In [ ]:
!pip install -U langchain-ollama langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [ ]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [ ]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [ ]:
df.shape

(40000, 2)

In [ ]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [ ]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [ ]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [ ]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [ ]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [ ]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

CPU times: user 1.89 s, sys: 13.2 ms, total: 1.9 s
Wall time: 1.91 s


In [ ]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [ ]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.41 s, sys: 44.4 ms, total: 4.46 s
Wall time: 2.58 s


In [ ]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [ ]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [ ]:
coffee_reviews_df.shape

(4981, 2)

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
!ollama pull gemma3:270m

In [ ]:
!ollama list

NAME           ID              SIZE      MODIFIED               
gemma3:270m    e7d36fb2c3b3    291 MB    Less than a second ago    


In [ ]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊




In [ ]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-15T12:42:33.811759121Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm = OllamaLLM(model="gemma3:270m")


In [ ]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊



In [ ]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [ ]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [ ]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [ ]:
chain = prompt | llm

In [ ]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [ ]:
response

'```python\ndef find_sum(numbers):\n  """\n  หาผลรวมของตัวเลขในชุดที่กำหนด\n  """\n  try:\n    total = 0\n    for number in numbers:\n      total += number\n    return total\n  except Exception as e:\n    print(f"ข้อผิดพลาด: {e}")\n    return 0  # หรือค่าอื่นที่เหมาะสม\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5]\nsum = find_sum(numbers)\nprint(sum)\n```\n\n**คำอธิบายโค้ด:**\n\n1.  **`def find_sum(numbers):`**:  กำหนดชื่อ function ที่จะถูกเรียกใช้เพื่อหาผลรวมของตัวเลข\n2.  **`try...except`**:  เป็นการจัดการกับข้อผิดพลาดที่อาจเกิดขึ้นในการทำงานกับข้อมูลจำนวนมาก\n3.  **`total = 0`**:  เริ่มต้นค่า `total` ที่ 0\n4.  **`for number in numbers:`**:  ทำการวนซ้ำผ่านแต่ละตัวเลขในชุด `numbers`\n5.  **`total += number`**:  เพิ่มค่า `number` เข้าไปใน `total`  (ค่าเพิ่มเข้ามาใน `total`)\n6.  **`return total`**:  คืนค่า `total`  ที่ได้จากการวนซ้ำ\n7.  **`except Exception as e:`**:  หากเกิดข้อผิดพลาด (เช่น การลบตัวเลข) ให้ทำการจัดการกับข้อผิดพลาดโดยใช้ `except` block\n8.  **`print(f"ข้อผิดพลาด: 

In [ ]:
from IPython.display import display, Markdown

In [ ]:
display(Markdown(response))

```python
def find_sum(numbers):
  """
  หาผลรวมของตัวเลขในชุดที่กำหนด
  """
  try:
    total = 0
    for number in numbers:
      total += number
    return total
  except Exception as e:
    print(f"ข้อผิดพลาด: {e}")
    return 0  # หรือค่าอื่นที่เหมาะสม

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5]
sum = find_sum(numbers)
print(sum)
```

**คำอธิบายโค้ด:**

1.  **`def find_sum(numbers):`**:  กำหนดชื่อ function ที่จะถูกเรียกใช้เพื่อหาผลรวมของตัวเลข
2.  **`try...except`**:  เป็นการจัดการกับข้อผิดพลาดที่อาจเกิดขึ้นในการทำงานกับข้อมูลจำนวนมาก
3.  **`total = 0`**:  เริ่มต้นค่า `total` ที่ 0
4.  **`for number in numbers:`**:  ทำการวนซ้ำผ่านแต่ละตัวเลขในชุด `numbers`
5.  **`total += number`**:  เพิ่มค่า `number` เข้าไปใน `total`  (ค่าเพิ่มเข้ามาใน `total`)
6.  **`return total`**:  คืนค่า `total`  ที่ได้จากการวนซ้ำ
7.  **`except Exception as e:`**:  หากเกิดข้อผิดพลาด (เช่น การลบตัวเลข) ให้ทำการจัดการกับข้อผิดพลาดโดยใช้ `except` block
8.  **`print(f"ข้อผิดพลาด: {e}")`**:  แสดงข้อผิดพลาดที่เกิดขึ้น
9.  **`return 0`**:  คืนค่าค่า `0`  ที่เหมาะสมเมื่อข้อผิดพลาดเกิดขึ้น
10. **`numbers = [1, 2, 3, 4, 5]`**:  สร้างชุดข้อมูลใหม่ (list)  ที่มีตัวเลข 1 ถึง 10
11. **`sum = find_sum(numbers)`**:  เรียกใช้ fungsi `find_sum` เพื่อหาผลรวมของตัวเลขในชุดข้อมูล `numbers`
12. **`print(sum)`**:  แสดงผลรวมของตัวเลขในชุดข้อมูล `numbers`

**ตัวอย่างการใช้งาน:**

```python
numbers = [1, 2, 3, 4, 5]
sum = find_sum(numbers)
print(sum)
```

**Output:**

```
15
```

**ข้อดีของโค้ดนี้:**

*   **การจัดการกับข้อผิดพลาด:**  โค้ดสามารถจัดการกับข้อผิดพลาดที่อาจเกิดขึ้นได้ เช่น การลบตัวเลข หรือการหมดค่า
*   **การใช้ `try...except`:**  โค้ดใช้ `try...except` เพื่อจัดการกับข้อผิดพลาดที่อาจเกิดขึ้น
*   **การใช้ `for` loop:**  โค้ดใช้ `for` loop เพื่อวนซ้ำให้ดีที่สุด
*   **การแสดงผลรวม:**  โค้ดแสดงผลรวมของตัวเลขในชุดข้อมูล

**ข้อควรระวัง:**

*   โค้ดนี้ไม่แนะนำให้ใช้โค้ดที่ซับซ้อนมากนัก เพราะอาจทำให้โค้ดทำงานได้ไม่ถูกต้อง
*   ควรตรวจสอบให้แน่ใจว่าตัวเลขที่ใช้ในการคำนวณมีค่าเป็นจำนวนเต็มก่อนการคำนวณ
*   ควรใช้ `try...except`  เพื่อจัดการกับข้อผิดพลาดที่อาจเกิดขึ้น



In [ ]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [ ]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [ ]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [ ]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [ ]:
parser_chain = StrOutputParser()

In [ ]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [ ]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [ ]:
result

'ถูกต้องแล้วครับ โจทย์ที่ถูกต้องคือ:\n\n**x = 3.125**'

In [ ]:
prompt_template = """
[1. Role/Context บอก LLM ว่าควรทำตัวเป็นอะไร]
คุณเป็น AI ที่เชี่ยวชาญในการวิเคราะห์รีวิวภาษาไทย

[2. Task Description บอกงานที่ต้องทำ]
จาก Review ต่อไปนี้ ให้ Extract 3 keywords
ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

[3. Input Data ข้อมูลที่ต้องประมวลผล]
Review: {review}

[4. Output Format บอกรูปแบบของคำตอบ]
ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}

[5. Constraints ข้อจำกัดหรือเงื่อนไข]
- ต้องเป็นคำภาษาไทย
- คั่นด้วย comma
- หากไม่พบให้ตอบ 'ไม่ระบุ'

[6. Examples (optional) ตัวอย่าง input/output]
ตัวอย่าง
Input: "กาแฟอร่อย บรรยากาศดี ราคาแพง"
Output: {"keywords": "กาแฟ, บรรยากาศ, ราคา"}
"""

In [ ]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [ ]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [ ]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [ ]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [ ]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [ ]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [ ]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 6.91 s, sys: 633 ms, total: 7.54 s
Wall time: 4min 32s


In [ ]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": ["ร้านอาหารใหญ่", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มันคาว", "ขนมไทย", "ขนมหวาน", "ฟรี", "ราคา", "ไม่ไหว", "แพง", "เกินไป", "รับไม่ไหว", "ว", "กินแล้ว", "ขนมหวาน", "ขนมไทย", "แนะนำ", "ร้านอาหาร", "ร้านค้า", "ร้านอาหารใหญ่", "ร้านอาหาร", "ร้านอาหารใหญ่มาก", "ร้านอาหารมาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้านอาหารใหญ่มาก", "ร้

In [ ]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [ ]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    5 minutes ago             


In [ ]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b:latest")

In [ ]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 95.9 ms, sys: 16 ms, total: 112 ms
Wall time: 16.7 s


In [ ]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะทำตามขั้นตอนที่คุณระบุมาเพื่อดึงคำสำคัญจากรีวิวนี้ครับ

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้พูดถึงประสบการณ์การทานอาหารที่ร้านอาหารแห่งหนึ่ง โดยมีประเด็นหลักๆ ดังนี้:

*   ร้านใหญ่มาก
*   ห้องน้ำอยู่ก่อนถึงห้องอาหาร
*   อาหารบางอย่าง (น้ำผึ้ง, ลาบไข่ต้ม, ทอดมันหัวปลี, พะแนงห่อไข่) มีรสชาติพอใช้
*   ราคาอาหารค่อนข้างสูง

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

หัวข้อหลักที่กล่าวถึงในรีวิวคือ:

*   ขนาดร้าน
*   คุณภาพอาหาร (โดยเฉพาะรสชาติ)
*   ราคา

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

จากหัวข้อหลักที่กล่าวถึง ผมเลือกคำเหล่านี้เป็น keywords ที่สำคัญที่สุด:

1.  ร้านใหญ่
2.  ราคาแพง
3.  อาหารไทย

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ร้านใหญ่", "ราคาแพง", "อาหารไทย"]
}
```

หวังว่าคำตอบนี้จะเป็นประโยชน์นะครับ หากมีอะไรให้ผมช่วยเพิ่มเติม บอกได้เลยครับ

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ย

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [ ]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [ ]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 163 ms, sys: 15 ms, total: 178 ms
Wall time: 16.1 s


In [ ]:
results[0]['text']['keywords']

['ราคา', 'รสชาติ', 'ขนาดร้าน']

In [ ]:
results[1]['text']['keywords']

['วาวี', 'กาแฟ', 'อารีย์']

In [ ]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 143 ms, sys: 15 ms, total: 158 ms
Wall time: 14.4 s


In [ ]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'รสชาติ', 'ขนาด'], ['วาวี', 'ลาเต้', 'บรรยากาศ'])

In [ ]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [ ]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [ ]:
inputs = [{"review": r} for r in sample_reviews]

In [ ]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 17.3 ms, sys: 1.94 ms, total: 19.3 ms
Wall time: 2.29 s


In [ ]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [ ]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 100

results_1000 = []

In [ ]:
from tqdm import tqdm

In [ ]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 10/10 [19:40<00:00, 118.04s/it]


In [ ]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, บรรยากาศคลาสสิก
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
Shibuya Honey Toast, ฮันนี่ โทสท์, ราคา
คาเฟ่, วานิลลา, คาโบนาร่า


In [ ]:
# Load coffee_reviews_df from DuckDB as it's needed for subsequent steps.
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

In [ ]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [ ]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

In [ ]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [ ]:
context_extract = context_prompt | llm3 | context_parser

In [ ]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [ ]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [ ]:
results_1000[0]['text']['keywords'].split(',')

['ราคา', ' บรรยากาศ', ' ไข่ต้ม']

In [ ]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 286946.98it/s]


In [ ]:
len(context_inputs)

3000

In [ ]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 1000/1000 [1:14:34<00:00,  4.47s/it]


In [ ]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ\nเขียวชอุ่ม\n\nว่าแล้วไม่รอช้

In [ ]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 425385.80it/s]


In [ ]:
len(sentiment_inputs)

3000

In [ ]:
sentiment_inputs[:6]

[{'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'บรรยากาศร้านใหญ่มากกกกกกก'},
 {'message': 'ไข่ต้ม'},
 {'message': 'กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด'},
 {'message': 'บรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี อย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ เขียวชอุ่ม'},
 {'message': 'รสชาติและกลิ่นมันไม่ธรรมดาเลย หอมๆ ถือว่าเป็นมวยรุ่นเดียวกะ Starbuck ได้'}]

In [ ]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [43:04<00:00,  2.58s/it]


In [ ]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม'],
  'sentiment': ['negative', 'positive', 'neutral']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล

In [ ]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'].split(','))},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [97]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')